In [ ]:
!pip3 install matplotlib > /dev/null 
!pip3 install ipympl > /dev/null 
!pip3 install ipywidgets > /dev/null
!pip3 install numpy > /dev/null
!pip3 install scipy > /dev/null
!pip3 install tabulate > /dev/null
%matplotlib widget
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:

import gzip
import numpy as np
from tabulate import tabulate
from lib.data_chain import DataChain
from lib.dataset import Dataset
from lib.ui.data_plotter_ui import DataPlotterUi
from lib.filters.ema_filter import EMAFilter
# from lib.filters.median_filter import MedianFilter
from lib.ui.data_trimmer_ui import DataTrimmerUi
from lib.file_manager import FileManager
from lib.ui.file_manager_ui import FileManagerUi
from lib.global_state import GobalState
from IPython.display import HTML, display

base_path = "./logs"
global_state = GobalState()

# Data Chain Processing
filter_chain = DataChain()
filter_chain.add_filter(EMAFilter(k_factor=0.05))

# Init data Trimmer
data_trimmer = DataTrimmerUi(max_depth = 2)
# Init File Selector
file_manager_ui = FileManagerUi(global_state, FileManager(base_path, prefix="DMM", extension="csv.gz"))

def read_file(file):
    with gzip.open(file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        return Dataset(np.array(lines)) 

def analize(dataset: Dataset):
    data_slice = dataset.slice()
    filtered_data_slice = np.array(filter_chain.apply(data_slice))

    dt = 1.0 / dataset.meta.get('sample_rate', 1.0)
    xvalues = dataset.data_range[0]*dt + np.arange(len(filtered_data_slice)) * dt
    data_plotter = DataPlotterUi(title="Оброблені данні", ylabel=dataset.meta.get('config', 'Values'))
    data_plotter.display(filtered_data_slice, xvalues = xvalues)
    
    stats = []
    stats.append(['Вихідні данні', dataset.data_range[0], dataset.data_range[1], dataset.data_range[1]-dataset.data_range[0], data_slice.mean(), data_slice.min(), data_slice.max(), data_slice.max()-data_slice.min()])
    stats.append(['Оброблені данні', dataset.data_range[0], dataset.data_range[1], dataset.data_range[1]-dataset.data_range[0], filtered_data_slice.mean(), filtered_data_slice.min(), filtered_data_slice.max(), filtered_data_slice.max()-filtered_data_slice.min()])
    display(HTML(tabulate(stats, headers=['Title', 'From', 'To', 'Span', 'Avg', 'Min', 'Max',  'Max-Min'], tablefmt='html')))

# Trimer Callback
data_trimmer.on_complete(analize)
# File Manager Callback
file_manager_ui.on_select(lambda file, *args: data_trimmer.display(read_file(file)))
file_manager_ui.display()